In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import copy


In [2]:
# 加载并预处理 CIFAR-100 数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# 将训练集划分为训练集和验证集
train_indices, val_indices = train_test_split(np.arange(len(trainset)), test_size=0.1, random_state=42)
train_subset = torch.utils.data.Subset(trainset, train_indices)
val_subset = torch.utils.data.Subset(trainset, val_indices)

trainloader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)
valloader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 定义 ResNet50 模型
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),  # Dropout layer for regularization
            nn.Linear(2048, 100)
        )

    def forward(self, x):
        return self.model(x)

resnet50 = ResNet50().cuda()

/home/asu/miniconda3/envs/dl_pytorch_hf/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asu/miniconda3/envs/dl_pytorch_hf/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/asu/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|███████████████████████████████████████| 97.8M/97.8M [06:30<00:00, 262kB/s]


In [4]:
# 训练函数
def train(model, trainloader, criterion, optimizer, device):
    global batch_count
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(trainloader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

In [5]:
# 验证函数
def validate(model, valloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(valloader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

In [6]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)

In [7]:
# 主训练循环
num_epochs = 200
device = torch.device('cuda')

for epoch in range(num_epochs):
    train_loss, train_acc = train(resnet50, trainloader, criterion, optimizer, device)
    val_loss, val_acc = validate(resnet50, valloader, criterion, device)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

Epoch 1/200, Train Loss: 3.1486, Train Acc: 25.19%, Val Loss: 2.0534, Val Acc: 45.20%
Epoch 2/200, Train Loss: 1.9206, Train Acc: 48.11%, Val Loss: 1.7232, Val Acc: 53.32%
Epoch 3/200, Train Loss: 1.4586, Train Acc: 59.23%, Val Loss: 1.5909, Val Acc: 56.36%
Epoch 4/200, Train Loss: 1.1333, Train Acc: 67.44%, Val Loss: 1.5802, Val Acc: 57.46%
Epoch 5/200, Train Loss: 0.8966, Train Acc: 73.61%, Val Loss: 1.6109, Val Acc: 58.56%
Epoch 6/200, Train Loss: 0.7083, Train Acc: 78.92%, Val Loss: 1.6350, Val Acc: 59.34%
Epoch 7/200, Train Loss: 0.5757, Train Acc: 82.67%, Val Loss: 1.7088, Val Acc: 58.84%
Epoch 8/200, Train Loss: 0.4568, Train Acc: 86.18%, Val Loss: 1.7796, Val Acc: 58.82%
Epoch 9/200, Train Loss: 0.3792, Train Acc: 88.46%, Val Loss: 1.8216, Val Acc: 58.78%
Epoch 10/200, Train Loss: 0.3195, Train Acc: 90.46%, Val Loss: 1.8679, Val Acc: 59.58%
Epoch 11/200, Train Loss: 0.2559, Train Acc: 92.21%, Val Loss: 1.9199, Val Acc: 59.60%
Epoch 12/200, Train Loss: 0.2295, Train Acc: 93.00%,

KeyboardInterrupt: 